# CAPSTONE PROJECT PHASE - 1

# Scrapping  Apartments.com

# FOR NEW JERSEY

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of New Jersey
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[9]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 15 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

690

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                    City                                       Address  \
0                 Edison                            Edison NJ \nEdison   
1                 Edison                            Edison NJ \nEdison   
2    Gloucester Township  Gloucester Township NJ \nGloucester Township   
3    Gloucester Township  Gloucester Township NJ \nGloucester Township   
4        McGinley Square              Jersey City NJ \nMcGinley Square   
..                   ...                                           ...   
572          Mantoloking                  Mantoloking NJ \nMantoloking   
573              Linwood                          Linwood NJ \nLinwood   
574         Margate City                Margate City NJ \nMargate City   
575           Westampton                    Westampton NJ \nWestampton   
576    Voorhees Township      Voorhees Township NJ \nVoorhees Township   

    StateCode                        Country Rental_Price_INR Deposit_INR  \
0           -               NJ 088

In [7]:
driver.quit()

# SCRAPPING FOR LAS VEGAS

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Las Vegas
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[4]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 15 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [7]:
len(product_urls)

1097

In [5]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Use regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                 City                                       Address StateCode  \
0           Las Vegas        W Meranto Ave Las Vegas NV \nLas Vegas         -   
1           Las Vegas        W Meranto Ave Las Vegas NV \nLas Vegas         -   
2     South Las Vegas  Bella Luna St Las Vegas NV \nSouth Las Vegas         -   
3     South Las Vegas  Bella Luna St Las Vegas NV \nSouth Las Vegas         -   
4       Peccole Ranch  Castle Crest Dr Las Vegas NV \nPeccole Ranch         -   
...               ...                                           ...       ...   
1041    Sunrise Manor                  Las Vegas NV \nSunrise Manor         -   
1042        Huntridge                      Las Vegas NV \nHuntridge         -   
1043    Spring Valley                  Las Vegas NV \nSpring Valley         -   
1044    Spring Valley                  Las Vegas NV \nSpring Valley         -   
1045    Lone Mountain                  Las Vegas NV \nLone Mountain         -   

                           

In [6]:
driver.quit()

# SCRAPPING FOR PHILADELPHIA

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Philadelphia
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[8]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=10
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

520

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


             City                                   Address StateCode  \
0      Kensington              Philadelphia PA \nKensington         -   
1      Kensington              Philadelphia PA \nKensington         -   
2      Kensington              Philadelphia PA \nKensington         -   
3      Kensington              Philadelphia PA \nKensington         -   
4    Point Breeze            Philadelphia PA \nPoint Breeze         -   
..            ...                                       ...       ...   
326   Walnut Hill     S th St Philadelphia PA \nWalnut Hill         -   
327   Grays Ferry  Gerritt St Philadelphia PA \nGrays Ferry         -   
328  Hunting Park            Philadelphia PA \nHunting Park         -   
329    Bridesburg              Philadelphia PA \nBridesburg         -   
330    Bridesburg              Philadelphia PA \nBridesburg         -   

                                 Country  Rental_Price_INR  Deposit_INR  \
0                   PA 19134\nKensington        

In [7]:
driver.quit()

# SCRAPPING FOR MICHIGAN

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Michigan
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[11]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

843

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df2 = pd.DataFrame(data)

# Displaying the DataFrame
print(df2)

# Saving the DataFrame to a CSV file
df2.to_csv('outputs.csv',index=False)


                   City                                            Address  \
0        Mount Pleasant                 Mount Pleasant MI \nMount Pleasant   
1        Mount Pleasant                 Mount Pleasant MI \nMount Pleasant   
2              Bay City                             Bay City MI \nBay City   
3              Bay City                             Bay City MI \nBay City   
4                  Vine                                Kalamazoo MI \nVine   
..                  ...                                                ...   
507  North Grand Rapids  Rockbluff Dr NE Comstock Park MI \nNorth Grand...   
508            Brighton               Huron Park Dr Brighton MI \nBrighton   
509           Royal Oak               Normandy Rd Royal Oak MI \nRoyal Oak   
510           Kalamazoo                   E O Ave Kalamazoo MI \nKalamazoo   
511           Mt Olivet                             Detroit MI \nMt Olivet   

    StateCode                                      Country Rent

In [ ]:
driver.quit()

# SCRAPPING FOR HOUSTON

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Houston
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[1]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

1089

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                    City                                           Address  \
0    Southbelt-Ellington  Chisholm Wood Ln Houston TX \nSouthbeltEllington   
1    Southbelt-Ellington  Chisholm Wood Ln Houston TX \nSouthbeltEllington   
2      Northwest Houston        Bach Elm St Houston TX \nNorthwest Houston   
3      Northwest Houston        Bach Elm St Houston TX \nNorthwest Houston   
4           Lake Houston       Sterling Point Ln Houston TX \nLake Houston   
..                   ...                                               ...   
918           South Park                           Houston TX \nSouth Park   
919             Eastwood                             Houston TX \nEastwood   
920       Meyerland Area                       Houston TX \nMeyerland Area   
921    Northwest Houston                    Houston TX \nNorthwest Houston   
922    Northwest Houston                    Houston TX \nNorthwest Houston   

    StateCode                                 Country Rental_Pr

In [7]:
driver.quit()

# SCRAPPING FOR ATLANTA

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [4]:
# For rentals of Atlanta
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[2]/a')
element.click()
time.sleep(2)

In [5]:
# scrapping all product urls
product_urls = []
start=0
end=10
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [6]:
len(product_urls)

630

In [7]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Create the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                  City                                       Address  \
0             Sandtown       Campbellton Rd SW Atlanta GA \nSandtown   
1             Sandtown       Campbellton Rd SW Atlanta GA \nSandtown   
2         Aerotropolis  Jesse A Dent Jr Rd Atlanta GA \nAerotropolis   
3         Aerotropolis  Jesse A Dent Jr Rd Atlanta GA \nAerotropolis   
4    Southeast Atlanta     Lois Ln SE Atlanta GA \nSoutheast Atlanta   
..                 ...                                           ...   
537       South Fulton                     Atlanta GA \nSouth Fulton   
538          Brookwood                        Atlanta GA \nBrookwood   
539       South Fulton                     Atlanta GA \nSouth Fulton   
540      Chastain Park                    Atlanta GA \nChastain Park   
541        Druid Hills                      Atlanta GA \nDruid Hills   

    StateCode                               Country Rental_Price_INR  \
0           -           Atlanta, GA 30331\nSandtown           1

In [ ]:
driver.quit()

# SCRAPPING FOR ORLANDO

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Orlando
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[7]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

824

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            if special_character_index != -1:  # If the special character is found
                state_code = re.findall(r'\d+', full_address[special_character_index + 1:])

                if state_code:
                    StateCode.append(state_code[0])
                else:
                    StateCode.append('-')  
            else:
                StateCode.append('-')

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:  # If the special character is found
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                City                                      Address StateCode  \
0       East Orlando  Cherry Valley Way Orlando FL \nEast Orlando         -   
1       East Orlando  Cherry Valley Way Orlando FL \nEast Orlando         -   
2       East Orlando        Pacifica Dr Orlando FL \nEast Orlando         -   
3       East Orlando        Pacifica Dr Orlando FL \nEast Orlando         -   
4       East Orlando      Tree Shore Dr Orlando FL \nEast Orlando         -   
..               ...                                          ...       ...   
762  Doctor Phillips                 Orlando FL \nDoctor Phillips         -   
763  Waterford Lakes     Harkness Ct Orlando FL \nWaterford Lakes         -   
764       Metro West                      Orlando FL \nMetro West         -   
765       Lake Tibet                      Orlando FL \nLake Tibet         -   
766        Lake Nona                       Orlando FL \nLake Nona         -   

                                Country Rental_Pric

In [7]:
driver.quit()

# SCRAPPING FOR LOS ANGELES

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Los Angeles
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[3]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 20 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

1159

In [7]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            state_code = re.findall(r'\d+', full_address)

            if state_code and state_code[0].isdigit() and len(state_code[0]) > 3:
                StateCode.append(state_code[0])
            else:
                StateCode.append('-')  

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                          City                                      Address  \
0                  Los Angeles  De Longpre Ave Los Angeles CA \nLos Angeles   
1                  Los Angeles  De Longpre Ave Los Angeles CA \nLos Angeles   
2                  Los Angeles  De Longpre Ave Los Angeles CA \nLos Angeles   
3                  Los Angeles  De Longpre Ave Los Angeles CA \nLos Angeles   
4             South Central LA    W th St Los Angeles CA \nSouth Central LA   
..                         ...                                          ...   
347                Los Angeles                 Los Angeles CA \nLos Angeles   
348                Los Angeles                 Los Angeles CA \nLos Angeles   
349                Los Angeles                 Los Angeles CA \nLos Angeles   
350  North San Fernando Valley   Los Angeles CA \nNorth San Fernando Valley   
351                Los Angeles                 Los Angeles CA \nLos Angeles   

    StateCode                                  Coun

In [8]:
driver.quit()

# SCRAPPING OF JACKSONVILLE

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Jacksonville
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[5]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 15 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

1208

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            state_code = re.findall(r'\d+', full_address)

            if state_code and state_code[0].isdigit() and len(state_code[0]) > 3:
                StateCode.append(state_code[0])
            else:
                StateCode.append('-')  

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                            City  \
0     Jacksonville Farms/Terrace   
1     Jacksonville Farms/Terrace   
2     Jacksonville Farms/Terrace   
3     Jacksonville Farms/Terrace   
4               Normandy Estates   
...                          ...   
1143               St. Augustine   
1144                 Orange Park   
1145                 Orange Park   
1146                 Orange Park   
1147                 Orange Park   

                                                Address StateCode  \
0     Tan Tara Trl Jacksonville FL \nJacksonville Fa...     32221   
1     Tan Tara Trl Jacksonville FL \nJacksonville Fa...     32221   
2     Tan Tara Trl Jacksonville FL \nJacksonville Fa...     32221   
3     Tan Tara Trl Jacksonville FL \nJacksonville Fa...     32221   
4          Spirit Ct Jacksonville FL \nNormandy Estates     32221   
...                                                 ...       ...   
1143           Brybar Dr St Augustine FL \nSt Augustine     32095   
1144           

In [7]:
driver.quit()

# SCRAPPING FOR NASHVILLE

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of Nashville
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[6]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 15 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

913

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            state_code = re.findall(r'\d+', full_address)

            if state_code and state_code[0].isdigit() and len(state_code[0]) > 3:
                StateCode.append(state_code[0])
            else:
                StateCode.append('-')  

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                    City                                         Address  \
0               Hillwood   Hillwood Private Cove Nashville TN \nHillwood   
1               Hillwood   Hillwood Private Cove Nashville TN \nHillwood   
2              Nashville       Carriage Park Dr Nashville TN \nNashville   
3              Nashville       Carriage Park Dr Nashville TN \nNashville   
4    Southeast Nashville  Vinewood Ct Nashville TN \nSoutheast Nashville   
..                   ...                                             ...   
807  Southeast Nashville              Nashville TN \nSoutheast Nashville   
808            Nashville                        Nashville TN \nNashville   
809       East Nashville          Grace St Nashville TN \nEast Nashville   
810          Haynes Area         Stokers Ln N Nashville TN \nHaynes Area   
811    Belmont/Hillsboro     Ashwood Ave Nashville TN \nBelmontHillsboro   

    StateCode                                   Country Rental_Price_INR  \
0       372

In [7]:
driver.quit()

# SCRAPPING FOR BOSTON

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of BOSTON
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[1]/ul/li[11]/a')
element.click()
time.sleep(2)

In [4]:
# scrapping all product urls
product_urls = []
start=0
end=15
for page in range(start,end):#for loop for scrapping 15 page
    url=driver.find_elements(By.XPATH,'//a[@class="property-link"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href")) #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH,'//a[@class="next "]')     
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

1442

In [6]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            state_code = re.findall(r'\d+', full_address)

            if state_code and state_code[0].isdigit() and len(state_code[0]) > 3:
                StateCode.append(state_code[0])
            else:
                StateCode.append('-')  

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                 City                                   Address StateCode  \
0      St Elizabeth's  Wallingford Rd Boston MA \nSt Elizabeths     02135   
1      St Elizabeth's  Wallingford Rd Boston MA \nSt Elizabeths     02135   
2      Columbia Point   Crescent Ave Boston MA \nColumbia Point     02125   
3      Columbia Point   Crescent Ave Boston MA \nColumbia Point     02125   
4    South End Boston              Boston MA \nSouth End Boston     02118   
..                ...                                       ...       ...   
561   Egleston Square     School St Boston MA \nEgleston Square     02119   
562    Columbia Point     Newport St Boston MA \nColumbia Point     02125   
563    Columbia Point     Newport St Boston MA \nColumbia Point     02125   
564     Uphams Corner                 Boston MA \nUphams Corner     02125   
565     Uphams Corner                 Boston MA \nUphams Corner     02125   

                               Country  Rental_Price_INR  Deposit_INR  \
0 

In [7]:
driver.quit()

# SCRAPPING FOR NEW ORLEANS

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
# For rentals of NEW ORLEANS
element = driver.find_element(By.XPATH, '/html/body/div[1]/div/section[8]/section/div/div[2]/ul/li[10]/a')
element.click()
time.sleep(2)

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initializing an empty list to store product URLs
product_urls = []

# Defining the start and end page range
start = 0
end =5

# Iterating through each page in the specified range
for page in range(start, end):
    # Finding all product URLs on the current page
    url_elements = driver.find_elements(By.XPATH, '//a[@class="property-link"]')

    # Iterating through each URL element and appending the URL to the list
    for element in url_elements:
        product_urls.append(element.get_attribute("href"))

    # Checking if the current page is the last page
    if page == end - 1:
        # Exiting the loop if it's the last page
        break

    # Clicking the next button to navigate to the next page
    nxt_button = driver.find_element(By.XPATH, '//a[@class="next "]')
    nxt_button.click()

    # Waiting for the next page to load completely by checking staleness of the last URL element
    WebDriverWait(driver, 10).until(
        EC.staleness_of(url_elements[-1])
    )

    # Optionally, adding a sleep to ensure the page has fully loaded
    time.sleep(2)

# Displaying the number of unique URLs scraped after the loop completes
print("Number of unique URLs:", len(set(product_urls)))


Number of unique URLs: 163


In [6]:
len(product_urls)

217

In [7]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

special_character = ','
max_wait_time = 2
# Conversion rate: 1 USD = 83.12 INR
conversion_rate = 83.12

# Initializing lists to store data
City = []
Address = []
StateCode = []
Country = []
Rental_Price_INR = []
Deposit_INR = []
Bedrooms = []
Bathrooms = []
Area = []
Pets_Allowed = []
Washer_Dryer = []
AC = []
Parking = []
Fireplace = []
Dishwasher = []
Hardwood_Floors = []
Roof_Deck = []
Storage = []

# Loop through each product URL
for url in product_urls:
    try:
        driver.get(url)
        WebDriverWait(driver, max_wait_time).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="property-link"]'))
        )
        time.sleep(2)

        # Extracting City
        try:
            city_element = driver.find_element(By.XPATH, '//span[@class="neighborhoodAddress"]//a')
            full_address = city_element.text

            # Extracting text only before the special character
            city = full_address.split(special_character)[0].strip()

            City.append(city)
        except NoSuchElementException:
            City.append('-')

        # Extracting Address
        try:
            address_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')

            # Using regular expression to extract only text
            address_text = re.sub('[^a-zA-Z\s]', '', address_element.text)

            Address.append(address_text.strip())

        except NoSuchElementException:
            Address.append('-')

        # Extracting state_code
        try:
            state_code_element = driver.find_element(By.XPATH, '//span[@class="stateZipContainer"]')
            full_address = state_code_element.text

            state_code = re.findall(r'\d+', full_address)

            if state_code and state_code[0].isdigit() and len(state_code[0]) > 3:
                StateCode.append(state_code[0])
            else:
                StateCode.append('-')  

        except NoSuchElementException:
            StateCode.append('-')

        # Extracting Country
        try:
            country_element = driver.find_element(By.XPATH, '//div[@class="propertyAddressRow"]')
            full_address = country_element.text

            # Finding the index of the special character
            special_character_index = full_address.find(special_character)

            # Extracting text after the special character
            if special_character_index != -1:
                country = full_address[special_character_index + 1:].strip()
                Country.append(country)
            else:
                Country.append('USA')

        except NoSuchElementException:
            Country.append('USA')

        # Extracting Rental_Price_INR
        def convert_to_inr(price_usd):
            try:
                # Removing special characters and conversion to integer
                price_inr = int(''.join(filter(str.isdigit, price_usd)))

                # Converting into INR
                price_inr = round(price_inr * conversion_rate)

                return price_inr
            except ValueError:
                return None

        try:
            rental_price_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            rental_price_text = rental_price_element.text

            # Removing special characters
            rental_price_inr = convert_to_inr(rental_price_text)

            if rental_price_inr is not None:
                Rental_Price_INR.append(rental_price_inr)
            else:
                Rental_Price_INR.append('-')

        except NoSuchElementException:
            Rental_Price_INR.append('-')

        # Extracting Deposit
        def convert_to_inr(amount_usd):
            try:
                # Removing special characters and conversion to integer
                amount_inr = int(''.join(filter(str.isdigit, amount_usd)))

                # Converting into INR
                amount_inr = round(amount_inr * conversion_rate)

                return amount_inr
            except ValueError:
                return None

        try:
            deposit_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[1]//div//p[2]')
            deposit_text = deposit_element.text

            # Removing special characters and convert to INR
            deposit_inr = convert_to_inr(deposit_text)

            if deposit_inr is not None:
                Deposit_INR.append(deposit_inr)
            else:
                Deposit_INR.append('-')

        except NoSuchElementException:
            Deposit_INR.append('-')

        # Extracting Bedrooms
        try:
            bedrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[2]//div//p[2]')
            bedrooms_text = bedrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bedrooms_number = re.findall(r'\d+', bedrooms_text)

            if bedrooms_number:
                Bedrooms.append(bedrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bedrooms.append('-')

        except NoSuchElementException:
            Bedrooms.append('-')

        # Extracting Bathrooms
        try:
            bathrooms_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[2]')
            bathrooms_text = bathrooms_element.text

            # Using regular expression to extract only numbers without special characters
            bathrooms_number = re.findall(r'\d+', bathrooms_text)

            if bathrooms_number:
                Bathrooms.append(bathrooms_number[0])  # Using the first element if there are multiple matches
            else:
                Bathrooms.append('-')

        except NoSuchElementException:
            Bathrooms.append('-')

        # Extracting Area
        try:
            area_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[4]//div//p[2]')
            area_text = area_element.text

            # Using regular expression to extract only numbers without special characters
            area_number = re.findall(r'\d+', area_text)

            if area_number:
                Area.append(area_number[0])  # Using the first element if there are multiple matches
            else:
                Area.append('-')

        except NoSuchElementException:
            Area.append('-')

        # Extracting Pets_Allowed
        try:
            pets_allowed_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[4]//span')
            pets_allowed_text = pets_allowed_element.text.lower()

            # Mapping the text to "Yes" or "No"
            pets_allowed_result = 'yes' if pets_allowed_text else 'no'

            Pets_Allowed.append(pets_allowed_result)

        except NoSuchElementException:
            Pets_Allowed.append('No')

        # Extracting Washer_Dryer
        try:
            washer_dryer_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[1]//span')

            # Checking if the text is present
            washer_dryer_result = 'Yes' if washer_dryer_element.text else 'No'

            Washer_Dryer.append(washer_dryer_result)

        except NoSuchElementException:
            Washer_Dryer.append('No')

        # Extracting AC
        try:
            ac_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[2]//span')

            # Checking if the text is present
            ac_result = 'Yes' if ac_element.text else 'No'

            AC.append(ac_result)

        except NoSuchElementException:
            AC.append('No')

        # Extracting Parking
        try:
            parking_element = driver.find_element(By.XPATH, '//ul[@class="priceBedRangeInfo"]//li[3]//div//p[1]')

            # Checking if the text is present
            parking_result = 'Yes' if parking_element.text else 'No'

            Parking.append(parking_result)

        except NoSuchElementException:
            Parking.append('No')

        # Extracting Fireplace
        try:
            fireplace_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[5]//span')

            # Checking if the text is present
            fireplace_result = 'Yes' if fireplace_element.text else 'No'

            Fireplace.append(fireplace_result)

        except NoSuchElementException:
            Fireplace.append('No')

        # Extracting Dishwasher
        try:
            dishwasher_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[3]//span')

            # Checking if the text is present
            dishwasher_result = 'Yes' if dishwasher_element.text else 'No'

            Dishwasher.append(dishwasher_result)

        except NoSuchElementException:
            Dishwasher.append('No')

        # Extracting Hardwood_Floors
        try:
            hardwood_floors_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[8]//span')

            # Checking if the text is present
            hardwood_floors_result = 'Yes' if hardwood_floors_element.text else 'No'

            Hardwood_Floors.append(hardwood_floors_result)

        except NoSuchElementException:
            Hardwood_Floors.append('No')

        # Extracting Roof_Deck
        try:
            roof_deck_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[9]//span')

            # Checking if the text is present
            roof_deck_result = 'Yes' if roof_deck_element.text else 'No'

            Roof_Deck.append(roof_deck_result)

        except NoSuchElementException:
            Roof_Deck.append('No')

        # Extracting Storage
        try:
            storage_element = driver.find_element(By.XPATH, '//div[@class="specGroup"]//ul//li[7]//span')

            # Checking if the text is present
            storage_result = 'Yes' if storage_element.text else 'No'

            Storage.append(storage_result)

        except NoSuchElementException:
            Storage.append('No')

    except TimeoutException:
        pass  # Do nothing and move to the next iteration of the loop

# Creating the DataFrame after the loop
data = {
    'City': City,
    'Address': Address,
    'StateCode': StateCode,
    'Country': Country,
    'Rental_Price_INR': Rental_Price_INR,
    'Deposit_INR': Deposit_INR,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Area': Area,
    'Pets_Allowed': Pets_Allowed,
    'Washer_Dryer': Washer_Dryer,
    'AC': AC,
    'Parking': Parking,
    'Fireplace': Fireplace,
    'Dishwasher': Dishwasher,
    'Hardwood_Floors': Hardwood_Floors,
    'Roof_Deck': Roof_Deck,
    'Storage': Storage
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

# Saving the DataFrame to a CSV file
df.to_csv('outputs.csv', mode='a', header=False, index=False)


                      City                               Address StateCode  \
0                  Audubon   Palmer Ave New Orleans LA \nAudubon     70118   
1                  Audubon   Palmer Ave New Orleans LA \nAudubon     70118   
2                  Audubon     State St New Orleans LA \nAudubon     70118   
3                  Audubon     State St New Orleans LA \nAudubon     70118   
4      Read Boulevard East  New Orleans LA \nRead Boulevard East     70128   
..                     ...                                   ...       ...   
141  River Ridge/Jefferson   Jefferson LA \nRiver RidgeJefferson     70121   
142               Westbank                  Gretna LA \nWestbank     70053   
143               Westbank               Terrytown LA \nWestbank     70056   
144               Westbank                  Gretna LA \nWestbank     70053   
145                  Arabi                      Arabi LA \nArabi     70032   

                             Country  Rental_Price_INR  Deposit

In [8]:
import pandas as pd

# Reading the CSV file into a pandas DataFrame
df = pd.read_csv('outputs.csv')
df

,City,Address,StateCode,Country,Rental_Price_INR,Deposit_INR,Bedrooms,Bathrooms,Area,Pets_Allowed,Washer_Dryer,AC,Parking,Fireplace,Dishwasher,Hardwood_Floors,Roof_Deck,Storage
0,Mount Pleasant,Mount Pleasant MI \nMount Pleasant,-,MI 48858\nMount Pleasant,126675,126675,3,2,1,yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes
1,Bay City,Bay City MI \nBay City,-,MI 48706\nBay City,85198,85198,2,2,800,No,Yes,Yes,Yes,No,Yes,No,No,No
2,Vine,Kalamazoo MI \nVine,-,MI 49007\nVine,34910,34910,6,2,-,yes,Yes,Yes,Yes,Yes,Yes,No,No,No
3,Eastown,Grand Rapids MI \nEastown,-,MI 49506\nEastown,137148,137148,3,1,-,yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
4,Wyoming,Wyoming MI \nWyoming,-,MI 49509\nWyoming,166240,166240,3,1,1,yes,Yes,Yes,Yes,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5034,River Ridge/Jefferson,Jefferson LA \nRiver RidgeJefferson,70121,LA 70121\nRiver Ridge/Jefferson,166240,166240,3,2,1,No,No,No,Yes,No,No,No,No,No
5035,Westbank,Gretna LA \nWestbank,70053,LA 70053\nWestbank,145460,145460,3,1,1,No,Yes,Yes,Yes,No,No,No,No,No
5036,Westbank,Terrytown LA \nWestbank,70056,LA 70056\nWestbank,149616,149616,4,2,1,No,No,No,Yes,No,No,No,No,No
5037,Westbank,Gretna LA \nWestbank,70053,LA 70053\nWestbank,195332,195332,4,1,1,No,No,No,Yes,No,No,No,No,No


# PHASE 1 COMPLETED